In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Core

> Core utilities and classes for OligoSeeker

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, List, Tuple, Set
from collections import Counter
import re
from Bio.Seq import Seq

## Type Definitions

In [ ]:
#| export
# Type aliases
OligoCounter = Dict[int, Counter]
FastqPair = Tuple[str, str]

## DNA Utilities

In [ ]:
#| export
# Constants for complement mapping
COMPLEMENT = {
    'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A',
    '[': ']', ']': '[', '(': ')', ')': '(',
    '.': '.', 'N': 'N'
}

In [ ]:
#| export
class DNAUtils:
    """Utility class for DNA sequence operations."""
    
    @staticmethod
    def reverse_complement(sequence: str) -> str:
        """Generate reverse complement of a DNA sequence.
        
        Args:
            sequence: DNA sequence string
            
        Returns:
            Reverse complement of the input sequence
        """
        return ''.join(COMPLEMENT.get(char, char) for char in reversed(sequence))
    
    @staticmethod
    def validate_oligos(oligos: List[str]) -> Tuple[bool, Set[str]]:
        """Validate a list of oligo sequences.
        
        Args:
            oligos: List of oligo sequences to validate
            
        Returns:
            Tuple of (is_valid, invalid_characters)
        """
        invalid_chars = set()
        for oligo in oligos:
            for b in oligo:
                if b not in COMPLEMENT:
                    invalid_chars.add(b)
        return len(invalid_chars) == 0, invalid_chars

## Oligo Regular Expression Matching

In [ ]:
#| export
class OligoRegex:
    """Compiles and manages regex patterns for oligo searching."""
    
    def __init__(self, oligo: str):
        """Initialize regex patterns for forward and reverse oligo sequences.
        
        Args:
            oligo: Oligo sequence with potential NNN codon sites
        """
        self.oligo = oligo
        self.forward = re.compile(oligo.replace('NNN', '(...)'))  # Captures the codon in group 1
        rev_oligo = DNAUtils.reverse_complement(oligo)
        self.reverse = re.compile(rev_oligo.replace('NNN', '(...)'))  # Captures the codon in group 1
    
    
    def find_codon(self, read_1: str, read_2: str) -> str:
        """Find codon in paired reads.
        
        Args:
            read_1: Sequence from the first read
            read_2: Sequence from the second read
            
        Returns:
            The codon found or 'none' if not found
        """
        # Combine reads with a unique separator for pattern matching
        combined_read = f"{read_1}@{read_2}"
        
        # Try matching in forward direction
        match = self.forward.search(combined_read)
        if match:
            return match.group(1)
        
        # Try matching in reverse direction
        match = self.reverse.search(combined_read)
        if match:
            return str(Seq(match.group(1)).reverse_complement())
        
        # No match found
        return 'none'

In [ ]:
# Example oligo design for targeting position 42 in a gene
oligo_template = "GAACGTTATCCGCGTNNNACGTTCGAAGCTGGT"
#                 ^^^^^^^^^^^^^^^^ ^^^ ^^^^^^^^^^^^^^^^
#                 5' targeting     |   3' targeting
#                                 codon

# Create the regex pattern for finding this oligo in sequencing data
oligo_regex = OligoRegex(oligo_template)

# Example sequencing read pairs
read1 = "ATCGAACGTTATCCGCGTATGACGTTCGAAGCTGGTCG"
#                          ^^^
#                          codon = ATG (Met)
read2 = "CGATCGGTTCGAACGTCTTCACAGCATTG"

# Find the codon
found_codon = oligo_regex.find_codon(read1, read2)
print(f"Found codon: {found_codon}")  # Should print "ATG"

# Translate to amino acid
amino_acid = str(Seq(found_codon).translate())
print(f"Amino acid: {amino_acid}")  

Found codon: ATG
Amino acid: M


In [ ]:
def test_oligo_regex():
    """Test the OligoRegex class for finding codons in reads."""
    
    # Test 1: Basic functionality with forward match
    oligo = "GAACNNNCAT"
    regex = OligoRegex(oligo)
    read1 = "ACGTGAACATGCATTGC"  # Contains GAACATGCAT with ATG as the codon
    read2 = "GCAACTGTAGCGTACGT"  # No match
    
    codon = regex.find_codon(read1, read2)
    print(f"Test 1: Forward match - Oligo: {oligo}, Codon found: {codon}")
    expected = "ATG"
    if codon == expected:
        print(f"✓ Correctly found codon {expected} in forward direction")
    else:
        print(f"✗ Expected {expected}, got {codon}")
    
    # Test 2: Match in second read
    oligo = "GAACNNNCAT"
    regex = OligoRegex(oligo)
    read1 = "ACGTCGATCGATCG"  # No match
    read2 = "ACGTGAACCGGCATCG"  # Contains GAACCGGCAT with CGG as the codon
    
    codon = regex.find_codon(read1, read2)
    print(f"Test 2: Match in second read - Oligo: {oligo}, Codon found: {codon}")
    expected = "CGG"
    if codon == expected:
        print(f"✓ Correctly found codon {expected} in second read")
    else:
        print(f"✗ Expected {expected}, got {codon}")
    
    # Test 3: Reverse complement match
    oligo = "GAACNNNCAT"
    regex = OligoRegex(oligo)
    rev_comp = DNAUtils.reverse_complement(oligo.replace("NNN", "GTC"))  # ATGGTCGTTC
    read1 = "ACGTATGGTCGTTCGCA"  # Contains reverse complement with GTC as codon
    read2 = "GCAACTGTAGCGTACGT"  # No match
    
    codon = regex.find_codon(read1, read2)
    print(f"Test 3: Reverse complement match - Oligo: {oligo}, Codon found: {codon}")
    expected = "GAC"  # Reverse complement of GTC
    if codon == expected:
        print(f"✓ Correctly found reverse complement codon {expected}")
    else:
        print(f"✗ Expected {expected}, got {codon}")
        
    # Test 5: No match
    oligo = "GAACNNNCAT"
    regex = OligoRegex(oligo)
    read1 = "ACGTCGATCGATCG"  # No match
    read2 = "GCAACTGTAGCGTACGT"  # No match
    
    codon = regex.find_codon(read1, read2)
    print(f"Test 5: No match - Oligo: {oligo}, Result: {codon}")
    if codon == "none":
        print("✓ Correctly returned 'none' for no match")
    else:
        print(f"✗ Expected 'none', got {codon}")

# Run the tests
test_oligo_regex()

Test 1: Forward match - Oligo: GAACNNNCAT, Codon found: ATG
✓ Correctly found codon ATG in forward direction
Test 2: Match in second read - Oligo: GAACNNNCAT, Codon found: CGG
✓ Correctly found codon CGG in second read
Test 3: Reverse complement match - Oligo: GAACNNNCAT, Codon found: GAC
✓ Correctly found reverse complement codon GAC
Test 5: No match - Oligo: GAACNNNCAT, Result: none
✓ Correctly returned 'none' for no match


## Oligo Loading and Validation

In [ ]:
#| export
class OligoLoader:
    """Loads and validates oligo sequences from different sources."""
    
    @staticmethod
    def from_file(filepath: str) -> List[str]:
        """Load oligos from a file.
        
        Args:
            filepath: Path to the file containing oligo sequences (one per line)
            
        Returns:
            List of oligo sequences
            
        Raises:
            ValueError: If the file contains duplicate oligos or invalid characters
        """
        with open(filepath) as f:
            oligos = [line.strip() for line in f if line.strip()]
        return OligoLoader.validate_oligos(oligos)
    
    @staticmethod
    def from_string(oligo_string: str, delimiter: str = ',') -> List[str]:
        """Load oligos from a delimited string.
        
        Args:
            oligo_string: String containing oligo sequences
            delimiter: Delimiter character (default: comma)
            
        Returns:
            List of oligo sequences
            
        Raises:
            ValueError: If the string contains duplicate oligos or invalid characters
        """
        oligos = [o.strip() for o in oligo_string.split(delimiter) if o.strip()]
        return OligoLoader.validate_oligos(oligos)
    
    @staticmethod
    def validate_oligos(oligos: List[str]) -> List[str]:
        """Validate a list of oligo sequences.

        Args:
            oligos: List of oligo sequences to validate

        Returns:
            The validated list of oligos

        Raises:
            ValueError: If duplicate oligos found, invalid characters detected, or NNN pattern issues
        """
        # Check for duplicates
        if len(oligos) != len(set(oligos)):
            raise ValueError("Duplicate oligos found")

        # Check for valid characters
        is_valid, invalid_chars = DNAUtils.validate_oligos(oligos)
        if not is_valid:
            invalid_chars_str = ','.join(invalid_chars)
            raise ValueError(f"Invalid characters found in oligos: {invalid_chars_str}")

        # Check for NNN pattern - each oligo must contain exactly one NNN codon
        invalid_oligos = []
        for oligo in oligos:
            nnn_count = oligo.count('NNN')
            if nnn_count != 1:
                raise ValueError(f"{oligo} (contains {nnn_count} NNN patterns, must have exactly 1)")
                

        if invalid_oligos:
            error_msg = "Invalid NNN pattern in oligos:\n" + "\n".join(invalid_oligos)
            raise ValueError(error_msg)

        return oligos

In [ ]:
def test_oligo_loader_single_oligo_validation():
    """Test OligoLoader validation with a single oligo per test."""
   
    # Test 1: Valid oligo with one NNN pattern
    try:
        OligoLoader.validate_oligos(["GCGGATTACATTNNNAAATAACATCGT"])
        print("✓ Valid oligo with one NNN passed validation")
    except ValueError as e:
        print(f"✗ Valid oligo failed validation: {str(e)}")
    
    # Test 2: Oligo with no NNN pattern
    try:
        OligoLoader.validate_oligos(["GCGGATTACATTGCTAAATAACATCGT"])
        print("✗ Failed to detect missing NNN pattern")
    except ValueError as e:
        print(f"✓ Correctly detected missing NNN pattern: {str(e)}")
    
    # Test 3: Oligo with multiple NNN patterns
    try:
        OligoLoader.validate_oligos(["GCGGATTACATTNNNAAATAACNNNGT"])
        print("✗ Failed to detect multiple NNN patterns")
    except ValueError as e:
        print(f"✓ Correctly detected multiple NNN patterns: {str(e)}")
    
    # Test 4: Oligo with incorrect NNN format (NN instead of NNN)
    try:
        OligoLoader.validate_oligos(["TGACNNTAG"])
        print("✗ Failed to detect incorrect NN format")
    except ValueError as e:
        print(f"✓ Correctly detected incorrect NN format: {str(e)}")
    
    # Test 5: Oligo with invalid characters
    try:
        OligoLoader.validate_oligos(["TGACNNNXYZ"])
        print("✗ Failed to detect invalid characters")
    except ValueError as e:
        print(f"✓ Correctly detected invalid characters: {str(e)}")
    
    print("\nSingle oligo validation tests completed")

# Run the tests
test_oligo_loader_single_oligo_validation()

✓ Valid oligo with one NNN passed validation
✓ Correctly detected missing NNN pattern: GCGGATTACATTGCTAAATAACATCGT (contains 0 NNN patterns, must have exactly 1)
✓ Correctly detected multiple NNN patterns: GCGGATTACATTNNNAAATAACNNNGT (contains 2 NNN patterns, must have exactly 1)
✓ Correctly detected incorrect NN format: TGACNNTAG (contains 0 NNN patterns, must have exactly 1)
✓ Correctly detected invalid characters: Invalid characters found in oligos: X,Z,Y

Single oligo validation tests completed


In [ ]:
#OligoLoader.validate_oligos(["GCGGATTACATTGCTAAATAACATCGT"])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()